# Packages, dependencies, extras

In [1]:
#packages
from IPython.display import clear_output
#!pip install schedule

#from config import rh_username,rh_password
from my_config import rh_username,rh_password
import pandas_datareader.data as web
from datetime import datetime
import robin_stocks as r
import schedule,time
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

clear_output()
pd.set_option('display.max_rows', None)

In [2]:
#Add color for a bit of class.
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
 
#USAGE
# f"{bcolors.OKGREEN}STRING\n{bcolors.ENDC}"

# Robinhood login & functionality

In [ ]:
#https://robin-stocks.readthedocs.io/en/latest/
login = r.authentication.login(username=rh_username,password=rh_password,store_session=True)
access_token=login['access_token']
token_type=login['token_type']

In [4]:
#Crypto functions for reference.
"""Contains functions to get information about crypto-currencies."""
import robin_stocks.helper as helper
import robin_stocks.urls as urls

@helper.login_required
def load_crypto_profile(info=None):
    """Gets the information associated with the crypto account.
    :param info: The name of the key whose value is to be returned from the function.
    :type info: Optional[str]
    :returns: [dict] The function returns a dictionary of key/value pairs. \
    If a string is passed in to the info parameter, then the function will return \
    a string corresponding to the value of the key whose name matches the info parameter.
    :Dictionary Keys: * apex_account_number
                      * created_at
                      * id
                      * rhs_account_number
                      * status
                      * status_reason_code
                      * updated_at
                      * user_id
    """
    url = urls.crypto_account()
    data = helper.request_get(url, 'indexzero')
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_positions(info=None):
    """Returns crypto positions for the account.
    :param info: Will filter the results to get a specific value.
    :type info: Optional[str]
    :returns: [list] Returns a list of dictionaries of key/value pairs for each option. If info parameter is provided, \
    a list of strings is returned where the strings are the value of the key that matches info.
    :Dictionary Keys: * account_id
                      * cost_basis
                      * created_at
                      * currency
                      * id
                      * quantity
                      * quantity_available
                      * quantity_held_for_buy
                      * quantity_held_for_sell
                      * updated_at
    """
    url = urls.crypto_holdings()
    data = helper.request_get(url, 'pagination')
    return(helper.filter_data(data, info))


def get_crypto_currency_pairs(info=None):
    """Gets a list of all the cypto currencies that you can trade.
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [list] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * asset_currency
                      * display_only
                      * id
                      * max_order_size
                      * min_order_size
                      * min_order_price_increment
                      * min_order_quantity_increment
                      * name
                      * quote_currency
                      * symbol
                      * tradability
    """
    url = urls.crypto_currency_pairs()
    data = helper.request_get(url, 'results')
    return(helper.filter_data(data, info))


def get_crypto_info(symbol, info=None):
    """Gets information about a crpyto currency.
    :param symbol: The crypto ticker.
    :type symbol: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then will return a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a strings representing the value of the key.
    :Dictionary Keys: * asset_currency
                      * display_only
                      * id
                      * max_order_size
                      * min_order_size
                      * min_order_price_increment
                      * min_order_quantity_increment
                      * name
                      * quote_currency
                      * symbol
                      * tradability
    """
    url = urls.crypto_currency_pairs()
    data = helper.request_get(url, 'results')
    data = [x for x in data if x['asset_currency']['code'] == symbol]
    if len(data) > 0:
        data = data[0]
    else:
        data = None
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_quote(symbol, info=None):
    """Gets information about a crypto including low price, high price, and open price
    :param symbol: The crypto ticker.
    :type symbol: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * ask_price
                      * bid_price
                      * high_price
                      * id
                      * low_price
                      * mark_price
                      * open_price
                      * symbol
                      * volume
 
    """
    id = get_crypto_info(symbol, info='id')
    url = urls.crypto_quote(id)
    data = helper.request_get(url)
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_quote_from_id(id, info=None):
    """Gets information about a crypto including low price, high price, and open price. Uses the id instead of crypto ticker.
    :param id: The id of a crypto.
    :type id: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [dict] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * ask_price
                      * bid_price
                      * high_price
                      * id
                      * low_price
                      * mark_price
                      * open_price
                      * symbol
                      * volume
    """
    url = urls.crypto_quote(id)
    data = helper.request_get(url)
    return(helper.filter_data(data, info))


@helper.login_required
def get_crypto_historicals(symbol, interval='hour', span='week', bounds='24_7', info=None):
    """Gets historical information about a crypto including open price, close price, high price, and low price.
    :param symbol: The crypto ticker.
    :type symbol: str
    :param interval: The time between data points. Can be '15second', '5minute', '10minute', 'hour', 'day', or 'week'. Default is 'hour'.
    :type interval: str
    :param span: The entire time frame to collect data points. Can be 'hour', 'day', 'week', 'month', '3month', 'year', or '5year'. Default is 'week'
    :type span: str
    :param bound: The times of day to collect data points. 'Regular' is 6 hours a day, 'trading' is 9 hours a day, \
    'extended' is 16 hours a day, '24_7' is 24 hours a day. Default is '24_7'
    :type bound: str
    :param info: Will filter the results to have a list of the values that correspond to key that matches info.
    :type info: Optional[str]
    :returns: [list] If info parameter is left as None then the list will contain a dictionary of key/value pairs for each ticker. \
    Otherwise, it will be a list of strings where the strings are the values of the key that corresponds to info.
    :Dictionary Keys: * begins_at
                      * open_price
                      * close_price
                      * high_price
                      * low_price
                      * volume
                      * session
                      * interpolated
                      * symbol
    """
    interval_check = ['15second', '5minute', '10minute', 'hour', 'day', 'week']
    span_check = ['hour', 'day', 'week', 'month', '3month', 'year', '5year']
    bounds_check = ['24_7', 'extended', 'regular', 'trading']

    if interval not in interval_check:
        print(
            'ERROR: Interval must be "15second","5minute","10minute","hour","day",or "week"', file=helper.get_output())
        return([None])
    if span not in span_check:
        print('ERROR: Span must be "hour","day","week","month","3month","year",or "5year"', file=helper.get_output())
        return([None])
    if bounds not in bounds_check:
        print('ERROR: Bounds must be "24_7","extended","regular",or "trading"', file=helper.get_output())
        return([None])
    if (bounds == 'extended' or bounds == 'trading') and span != 'day':
        print('ERROR: extended and trading bounds can only be used with a span of "day"', file=helper.get_output())
        return([None])


    symbol = helper.inputs_to_set(symbol)
    id = get_crypto_info(symbol[0], info='id')
    url = urls.crypto_historical(id)
    payload = {'interval': interval,
               'span': span,
               'bounds': bounds}
    data = helper.request_get(url, 'regular', payload)

    histData = []
    cryptoSymbol = data['symbol']
    for subitem in data['data_points']:
        subitem['symbol'] = cryptoSymbol
        histData.append(subitem)

    return(helper.filter_data(histData, info))

# SuperTrend Formula

## Binance.US data

In [9]:
import ccxt
import schedule
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from datetime import datetime
import time

exchange = ccxt.binanceus({
    "apiKey": BINANCE_KEY,
    "secret": BINANCE_SECRET
})

#Binance data
bars = exchange.fetch_ohlcv('ETH/USDT', timeframe='5m', limit=100)
df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close','volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df

,timestamp,open,high,low,close,volume
0,2021-08-31 23:05:00,3401.17,3417.17,3401.17,3414.35,14.05993
1,2021-08-31 23:10:00,3416.85,3425.58,3414.38,3420.17,25.98234
2,2021-08-31 23:15:00,3419.83,3419.83,3414.05,3414.05,8.72203
3,2021-08-31 23:20:00,3417.14,3422.42,3416.13,3417.84,6.82287
4,2021-08-31 23:25:00,3421.79,3428.20,3421.79,3422.33,7.07486
5,2021-08-31 23:30:00,3426.33,3439.19,3425.80,3432.97,46.89283
6,2021-08-31 23:35:00,3432.16,3437.10,3430.55,3431.72,4.83817
7,2021-08-31 23:40:00,3430.91,3433.62,3425.17,3433.62,34.95794
8,2021-08-31 23:45:00,3432.54,3440.56,3432.05,3439.89,18.80113
9,2021-08-31 23:50:00,3440.54,3444.17,3436.29,3443.14,11.17175


## Robinhood data

In [6]:
#Rh data
#columns=['timestamp', 'open', 'high', 'low', 'close','volume']
df = pd.DataFrame.from_dict(r.crypto.get_crypto_historicals('ETH', interval='5minute', span='day', bounds='24_7', info=None)).drop(columns=['session','interpolated','symbol']).tail(100).reset_index(drop=True)
bar = []
for k,v in df.begins_at.items():
    #print(str(time.mktime(datetime.strptime(v,"%Y-%m-%dT%H:%M:%SZ").timetuple()))[:12].replace('.','00'),df.open_price[k],df.high_price[k],df.low_price[k],df.close_price[k],df.volume[k])
    bar.append([str(time.mktime(datetime.strptime(v,"%Y-%m-%dT%H:%M:%SZ").timetuple()))[:12].replace('.','00'),df.open_price[k],df.high_price[k],df.low_price[k],df.close_price[k],df.volume[k]])


df = pd.DataFrame(bar[:-1], columns=['timestamp', 'open', 'high', 'low', 'close','volume'])
df = df.apply(pd.to_numeric)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

df

,timestamp,open,high,low,close,volume
0,2021-09-01 05:55:00,3407.340,3423.270000,3402.370,3415.570,0
1,2021-09-01 06:00:00,3415.570,3421.785000,3397.910,3403.530,0
2,2021-09-01 06:05:00,3403.530,3424.170000,3397.590,3417.165,0
3,2021-09-01 06:10:00,3417.165,3432.605000,3412.330,3422.025,0
4,2021-09-01 06:15:00,3422.025,3428.345000,3409.900,3416.510,0
5,2021-09-01 06:20:00,3411.155,3429.230000,3410.795,3416.845,0
6,2021-09-01 06:25:00,3416.845,3435.580000,3411.010,3426.275,0
7,2021-09-01 06:30:00,3426.275,3444.480000,3420.610,3435.875,0
8,2021-09-01 06:35:00,3430.170,3444.930000,3426.400,3432.930,0
9,2021-09-01 06:40:00,3432.930,3440.990000,3419.105,3434.515,0


# SuperTrend application

In [10]:
!pip install ccxt
clear_output()

In [ ]:
#SuperTrend
def tr(data):
    data['previous_close'] = data['close'].shift(1)
    data['high-low'] = abs(data['high'] - data['low'])
    data['high-pc'] = abs(data['high'] - data['previous_close'])
    data['low-pc'] = abs(data['low'] - data['previous_close'])

    tr = data[['high-low', 'high-pc', 'low-pc']].max(axis=1)

    return tr

def atr(data, period):
    data['tr'] = tr(data)
    atr = data['tr'].rolling(period).mean()

    return atr

def supertrend(df, period=7, atr_multiplier=3):
    hl2 = (df['high'] + df['low']) / 2
    df['atr'] = atr(df, period)
    df['upperband'] = hl2 + (atr_multiplier * df['atr'])
    df['lowerband'] = hl2 - (atr_multiplier * df['atr'])
    df['in_uptrend'] = True

    for current in range(1, len(df.index)):
        previous = current - 1

        if df['close'][current] > df['upperband'][previous]:
            df['in_uptrend'][current] = True
        elif df['close'][current] < df['lowerband'][previous]:
            df['in_uptrend'][current] = False
        else:
            df['in_uptrend'][current] = df['in_uptrend'][previous]

            if df['in_uptrend'][current] and df['lowerband'][current] < df['lowerband'][previous]:
                df['lowerband'][current] = df['lowerband'][previous]

            if not df['in_uptrend'][current] and df['upperband'][current] > df['upperband'][previous]:
                df['upperband'][current] = df['upperband'][previous]
        
    return df

#Assumes we're not in a position.
in_position = False

def check_buy_sell_signals(df):
    global in_position

    #print("checking for buy and sell signals\n")
    last_row_index = len(df.index) - 1
    previous_row_index = last_row_index - 1
    print("Uptrend: ",df['in_uptrend'][last_row_index],"\n","Close Price: ",df['close'][last_row_index],"\n\n")

    if not df['in_uptrend'][previous_row_index] and df['in_uptrend'][last_row_index]:
        print("Buy")
        if not in_position:
            #order = exchange.create_market_buy_order('ETH/USD', 0.05)
            #print(order)
            print("Bought")
            in_position = True
        else:
            print("already in position, nothing to do")
    
    if df['in_uptrend'][previous_row_index] and not df['in_uptrend'][last_row_index]:
        if in_position:
            print("Sell")
            #order = exchange.create_market_sell_order('ETH/USD', 0.05)
            #print(order)
            print("Sold")
            in_position = False
        else:
            print("You aren't in position, nothing to sell")

def run_bot():
    print(f"Fetching new data for {datetime.now().isoformat()}")
    df = pd.DataFrame.from_dict(r.crypto.get_crypto_historicals('DOGE', interval='5minute', span='week', bounds='24_7', info=None)).drop(columns=['session','interpolated','symbol']).tail(100).reset_index(drop=True)
    bar = []
    for k,v in df.begins_at.items():
        #print(str(time.mktime(datetime.strptime(v,"%Y-%m-%dT%H:%M:%SZ").timetuple()))[:12].replace('.','00'),df.open_price[k],df.high_price[k],df.low_price[k],df.close_price[k],df.volume[k])
        bar.append([str(time.mktime(datetime.strptime(v,"%Y-%m-%dT%H:%M:%SZ").timetuple()))[:12].replace('.','00'),df.open_price[k],df.high_price[k],df.low_price[k],df.close_price[k],df.volume[k]])
    df = pd.DataFrame(bar[:-1], columns=['timestamp', 'open', 'high', 'low', 'close','volume'])
    df = df.apply(pd.to_numeric)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    supertrend_data = supertrend(df)
    
    check_buy_sell_signals(supertrend_data)


schedule.every(10).seconds.do(run_bot)


while True:
    schedule.run_pending()
    time.sleep(1)

Fetching new data for 2021-09-01T00:29:02.712771
Uptrend:  True 
 Close Price:  0.279956 


Fetching new data for 2021-09-01T00:29:04.041202
Uptrend:  True 
 Close Price:  0.279956 


Fetching new bars for 2021-09-01T00:29:04.363508
Uptrend:  True 
 Close Price:  0.279956 


Fetching new bars for 2021-09-01T00:29:05.666423
Uptrend:  True 
 Close Price:  0.279987 


Fetching new bars for 2021-09-01T00:29:05.965769
Uptrend:  True 
 Close Price:  0.279987 


Fetching new data for 2021-09-01T00:29:07.240560
Uptrend:  True 
 Close Price:  0.279987 


Fetching new bars for 2021-09-01T00:29:07.543426
Uptrend:  True 
 Close Price:  0.279987 


Fetching new data for 2021-09-01T00:29:12.941760
Uptrend:  True 
 Close Price:  0.279956 


Fetching new data for 2021-09-01T00:29:14.264086
Uptrend:  True 
 Close Price:  0.279956 


Fetching new data for 2021-09-01T00:29:15.566262
Uptrend:  True 
 Close Price:  0.279987 


Fetching new bars for 2021-09-01T00:29:15.918693
Uptrend:  True 
 Close Price:  

Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:05.310655
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:06.644717
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:07.951025
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:31:09.329900
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:10.612300
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:31:11.921178
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:31:13.202264
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:31:14.500326
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:15.813869
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:31:17.279848
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:3

Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:33:05.148208
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:33:06.725456
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:33:08.640380
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:33:10.059664
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:33:11.395836
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:33:12.710827
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:33:14.053256
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:33:15.486266
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:33:16.825239
Uptrend:  True 
 Close Price:  0.280174 


Fetching new data for 2021-09-01T00:33:18.131402
Uptrend:  True 
 Close Price:  0.280174 


Fetching new bars for 2021-09-01T00:3

Fetching new data for 2021-09-01T00:35:04.036282
Uptrend:  True 
 Close Price:  0.279961 


Fetching new bars for 2021-09-01T00:35:05.638239
Uptrend:  True 
 Close Price:  0.279961 


Fetching new data for 2021-09-01T00:35:07.823400
Uptrend:  True 
 Close Price:  0.279961 


Fetching new data for 2021-09-01T00:35:08.604145
Uptrend:  True 
 Close Price:  0.279961 


Fetching new data for 2021-09-01T00:35:10.494619
Uptrend:  True 
 Close Price:  0.279961 


Fetching new bars for 2021-09-01T00:35:10.818973
Uptrend:  True 
 Close Price:  0.279961 


Fetching new bars for 2021-09-01T00:35:12.110943
Uptrend:  True 
 Close Price:  0.279961 


Fetching new bars for 2021-09-01T00:35:13.616589
Uptrend:  True 
 Close Price:  0.279961 


Fetching new data for 2021-09-01T00:35:15.121108
Uptrend:  True 
 Close Price:  0.279961 


Fetching new bars for 2021-09-01T00:35:17.421290
Uptrend:  True 
 Close Price:  0.279961 


Fetching new data for 2021-09-01T00:35:18.734964
Uptrend:  True 
 Close Price:  

In [ ]:
#En fin...